In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import importlib 

import colors, plot_func

In [ ]:
# k_stiff = np.array([1, 1])
# k_soft = np.array([1, 0.0001])
# thresh = np.array([40, 20])
# theta_ss = np.array([20, 20])
# init_buckle = np.array([1, 1])
# desired_buckle = np.array([-1, 1])

k_stiff = np.array([4.2, 2.0, 1.0, 1.0])
k_soft = np.array([0.2, 0.5, 0.1, 0.4])
thresh = np.array([30, 25, 15, 10])
theta_ss = np.array([25, 15, 10, 5])
init_buckle = np.array([-1, 1, -1, -1])
desired_buckle = np.array([1, -1, -1, 1])

# k_stiff = np.array([4.2, 2.0, 1.0])
# k_soft = np.array([0.2, 0.5, 0.1])
# thresh = np.array([30, 25, 15])
# theta_ss = np.array([5, 15, 10])
# init_buckle = np.array([-1, 1, 1])
# desired_buckle = np.array([1, 1, -1])

# k_stiff = np.array([1.0, 1.0])
# k_soft = np.array([1.0, 1.0])
# thresh = np.array([40, 20])
# theta_ss = np.array([0, 0])
# init_buckle = np.array([1, 1])
# desired_buckle = np.array([-1, -1])

supress_prints = False

problem = 'tau'

In [ ]:
iterations = 50
alpha = 0.025

In [ ]:
## Classes

class VariablesClass:
    """
    Class with variables dictated by supervisor
    """
    def __init__(self, k_stiff, k_soft, thresh, theta_ss):
        self.k_stiff = k_stiff
        self.k_soft = k_soft
        self.thresh = thresh
        self.theta_ss = theta_ss
        self.N_springs = np.size(k_stiff)
        
        # correct for un-physical threshold to move shim
        self.thresh[self.thresh<self.theta_ss] = self.theta_ss[self.thresh<self.theta_ss]
        
        # soft always 
        self.k_soft[self.k_soft>self.k_stiff] = self.k_stiff[self.k_soft>self.k_stiff]
    
    def set_normalizations(self, k_stiff, k_soft, theta_ss):
        self.k_bar = np.mean([np.mean(k_stiff), np.mean(k_soft)])
        self.theta_bar = np.mean(theta_ss)

        
class SupervisorClass:
    """
    Class with variables dictated by supervisor
    """
    def __init__(self, desired_buckle, alpha):
        self.desired_buckle = desired_buckle
        self.input_in_t = []
        self.input_update_in_t = []
        self.loss_in_t = []
        self.loss_MSE_in_t = []
        self.alpha = alpha
        
        self.input_update = 0
        self.input_update_in_t.append(self.input_update)
        
    def init_dataset(self, iterations, problem):
        self.iterations = iterations
        # self.theta_vec = np.array([-50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50])
        if problem == 'tau':
            self.theta_vec = np.random.uniform(-90, 90, iterations)
        elif problem == 'Fy':
            self.theta_1_vec = np.random.uniform(-90, 90, iterations)
            self.theta_2_vec = np.random.uniform(-90, 90, iterations)
            self.x_vec, self.y_vec = forward_points(L, self.theta_1_vec, self.theta_2_vec)
    
    def desired_tau(self):
        self.desired_tau_vec = np.zeros(np.size(self.theta_vec))
        for i, theta in enumerate(self.theta_vec):
            self.desired_tau_vec[i] = tau_hinge(theta, self.desired_buckle, Variabs)
            
    def set_theta(self, iteration):
        self.theta = self.theta_vec[iteration]
        self.desired_tau = self.desired_tau_vec[i]
        print('theta ', self.theta)
        
    def set_pos(self, iteration):
        self.x, self.y = self.x_vec[iteration], self.y_vec[iteration]
        self.desired_Fy = self.desired_Fy_vec[i]
        
    def calc_loss(self, tau):
        self.loss = loss(self.desired_tau, tau)
        self.loss_MSE = loss_MSE(self.loss)
        self.loss_in_t.append(self.loss)
        self.loss_MSE_in_t.append(self.loss_MSE)
        if not supress_prints:
            print('desired tau ', self.desired_tau)
            print('loss ', self.loss)
            print('MSE loss ', self.loss_MSE)
        
    def calc_input_update(self, State, Supervisor, Variabs):
        delta_theta = input_update(State.tau, Supervisor.loss, Supervisor.theta, Variabs.k_bar, Variabs.theta_bar)
        input_update_nxt = copy.copy(self.input_update) + self.alpha * delta_theta
        self.input_update = input_update_nxt
        self.input_update_in_t.append(self.input_update)
        if not supress_prints:
            print('delta theta', delta_theta)
            print('input_update ', self.input_update)


class StateClass:
    """
    Class with state variables
    """
    def __init__(self, Variabs, Supervisor, buckle=None):
        # self.buckle_in_t = np.zeros([Supervisor.iterations, Variabs.N_springs])
        # self.tau_in_t = np.zeros(Supervisor.iterations)
        self.buckle_in_t = []
        self.tau_in_t = []
        
        if buckle is not None:
            self.buckle = buckle
        else:
            self.buckle = np.ones(Variabs.N_springs)
        if not supress_prints:
            print('buckle pattern ', self.buckle)
        self.buckle_in_t.append(self.buckle)

    def calc_tau(self, theta, Variabs):
        self.tau = tau_hinge(theta, self.buckle, Variabs) 
        if not supress_prints:
            print('tau ', self.tau)
            
    def evolve_material(self, Supervisor, Variabs):
        buckle_nxt = np.zeros(Variabs.N_springs)
        for i in range(Variabs.N_springs):
            if self.buckle[i] == 1 and Supervisor.input_update > Variabs.thresh[i]:  # buckle left
                buckle_nxt[i] = -1
            elif self.buckle[i] == -1 and Supervisor.input_update < -Variabs.thresh[i]:  # buckle right
                buckle_nxt[i] = 1
            else:
                buckle_nxt[i] = self.buckle[i]
        self.buckle = copy.copy(buckle_nxt)
        self.buckle_in_t.append(self.buckle)
        if not supress_prints:
            print('buckle pattern ', self.buckle)       

In [ ]:
def tau_tot():
    return (np.sum(tau_hinge(theta, buckle, Variabs)**(-1)))**(-1)


def tau_hinge(theta, buckle, Variabs):
    return np.sum(tau_k(theta, buckle, Variabs))
    

def tau_k(theta, buckle, Variabs):
    tau_k = np.zeros(Variabs.N_springs)
    for i in range(Variabs.N_springs):
        if buckle[i] == 1 and theta > -Variabs.theta_ss[i] or buckle[i] == -1 and theta < Variabs.theta_ss[i]:
            k = Variabs.k_stiff[i]
        else:
            k = Variabs.k_soft[i]
        tau_k[i] = -k * (theta - (-buckle[i]) * Variabs.theta_ss[i])
    return tau_k


def loss(desired_tau, tau):
    return desired_tau - tau


def loss_MSE(loss):
    return np.mean(loss**2)


def input_update(tau, loss, theta, k_bar, theta_bar):
    tau_pre = 1
    theta_pre = -1
    # return (tau/k_bar-theta)*(loss)/(k_bar*theta_bar)
    # return (theta/theta_bar)*(loss)/(k_bar*theta_bar)
    return (tau_pre*tau/k_bar+theta_pre*theta)*(loss)/(k_bar*theta_bar)


def measure_full_response(Variabs, buckle, length=100):
    theta_vec = np.linspace(-180, 180, length)
    tau_vec = np.zeros([length])
    for i, theta in enumerate(theta_vec):
        tau_vec[i] = tau_hinge(theta, buckle, Variabs)
    return theta_vec, tau_vec

In [ ]:
def theta_from_xy(x, y, buckle, L=1.0):
    """
    Return (theta1, theta2) where:
      - theta2 is the relative elbow angle (same as before).
      - theta1 is the base angle measured from the VERTICAL (+y axis).
        If ccw_from_vertical=False, it will be clockwise-from-vertical.

    buckle = +1  -> elbow-up branch
    buckle = -1  -> elbow-down branch
    """
    # IK for 2R with L1=L2=L
    c2 = (x**2 + y**2 - 2*L**2) / (2*L**2)
    c2 = np.clip(c2, -1.0, 1.0)

    s2 = np.sqrt(max(0.0, 1.0 - c2**2))
    if buckle == 1:
        theta2 = np.arctan2(+s2, c2)
    else:
        theta2 = np.arctan2(-s2, c2)

    # q1 measured from +x (standard)
    q1_from_x = np.arctan2(y, x) - theta2/2
    theta1    = q1_from_x - np.pi/2

    return theta1, theta2


def Fy(theta1, theta2, tau1, tau2):
    return (np.sin(theta1)*tau2 + np.sin(theta1+theta2)*(tau2-tau1))/np.sin(theta2)


def forward_points(L, theta1, theta2):
    p0 = np.array([0.0, 0.0])
    p1 = p0 + L*np.array([-np.sin(theta1), np.cos(theta1)])
    p2 = p1 + L*np.array([-np.sin(theta1+theta2), np.cos(theta1+theta2)])
    return p0, p1, p2


def plot_arm(x, y, theta1, theta2):
    
    # Set the custom color cycle globally without cycler
    colors_lst, red, custom_cmap = colors.color_scheme()
    plt.rcParams['axes.prop_cycle'] = plt.cycler('color', colors_lst)
    
    p0, p1, p2 = forward_points(L, theta1, theta2)
    # N, tau1, tau2 = normal_force(L q1, q2, k1, k2, th1_ss, th2_ss)

    # Figure
    plt.figure(figsize=(4, 4))
    # Wall (horizontal at y=y_wall) and fixed x (vertical at x=x_star)
    xs = np.linspace(-2*L-0.5, 2*L+0.5, 2)
    plt.plot(xs, y*np.ones_like(xs), linewidth=2, linestyle='--', label="wall y=y_w")
    # plt.plot([x, x], [-(2*L+0.5), (2*L+0.5)], linewidth=1, linestyle=':', label="fixed x")

    # Links
    plt.plot([p0[0], p0[0]], [-L/3, p0[1]], '-k', linewidth=4)  # straight line up to first node 
    plt.plot([p0[0], p1[0]], [p0[1], p1[1]], linewidth=4)
    plt.plot([p1[0], p2[0]], [p1[1], p2[1]], linewidth=4)

    # Joints and tip
    plt.scatter([p0[0], p1[0], p2[0]], [p0[1], p1[1], p2[1]], s=60, zorder=3)
    # Annotate tip and normal force arrow
    plt.annotate("Tip", xy=(p2[0], p2[1]), xytext=(p2[0]+0.05, p2[1]+0.05))

    # Angle markers (small arcs)
    # base angle q1
    a = np.linspace(-np.pi/2, theta1+np.pi/2, 60)
    r = 0.15*L
    plt.plot(r*np.cos(a), r*np.sin(a))
    # elbow relative angle q2 centered at p1
    a2 = np.linspace(-np.pi/2+theta1, theta1+theta2+np.pi/2, 60)
    r2 = 0.15*L 
    plt.plot(p1[0] + r2*np.cos(a2), p1[1] + r2*np.sin(a2))
    plt.text(p1[0]+0.05, p1[1]+0.05, r"$\theta_2$")
    plt.text(0.05, 0.05, r"$\theta_1$")

    # Info box
#     info = (
#         f"theta1 = {np.degrees(theta1):.2f} deg\n"
#         f"theta2 = {np.degrees(theta2):.2f} deg\n"
#     )
#     plt.gca().text(0.02, 0.98, info, transform=plt.gca().transAxes,
#                    verticalalignment='top', bbox=dict(boxstyle='round', alpha=0.15))

    # Aesthetics
    plt.axis('equal')
    reach = 2*L + 0.4
    plt.xlim(-reach, reach)
    plt.ylim(-reach, reach)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.title(f"Tip (x, y)=({x:.2f}, {y:.2f})")
    plt.grid(True)
    # plt.legend(loc='lower left')
    plt.show()

In [ ]:
Variabs = VariablesClass(k_stiff, k_soft, thresh, theta_ss)
Variabs.set_normalizations(k_stiff, k_soft, theta_ss)

In [ ]:
Supervisor = SupervisorClass(desired_buckle, alpha)
Supervisor.init_dataset(iterations, problem)
Supervisor.desired_tau()

In [ ]:
State = StateClass(Variabs, Supervisor, buckle=init_buckle)

## Loop

## Measure tau - single hinge

In [ ]:
if problem == 'tau':
    theta_full, tau_full_init = measure_full_response(Variabs, State.buckle)
    theta_full, tau_full_desired  = measure_full_response(Variabs, desired_buckle)

    for i in range(iterations):

        ## Measure
        Supervisor.set_theta(i)
        State.calc_tau(Supervisor.theta, Variabs)

        ## Loss
        Supervisor.calc_loss(State.tau)

        ## Update
        Supervisor.calc_input_update(State, Supervisor, Variabs)
        State.calc_tau(Supervisor.input_update, Variabs)
        State.evolve_material(Supervisor, Variabs)

        theta_full, tau_full_final = measure_full_response(Variabs, State.buckle)
        plot_func.plot_response(theta_full, tau_full_init, tau_full_final, tau_full_desired, theta_range=[-90, 90], just_init=False)

    Supervisor.loss_MSE_norm_in_t = Supervisor.loss_MSE_in_t / (Variabs.k_bar * Variabs.theta_bar)

In [ ]:
if problem == 'Fy':
    importlib.reload(plot_func)

    plot_func.importants(State.buckle_in_t, Supervisor.desired_buckle, 
                         Supervisor.loss_MSE_norm_in_t, Supervisor.input_update_in_t)

    plot_func.plot_response(theta_full, tau_full_init, tau_full_desired, tau_full_desired, theta_range=[-90, 90], just_init=True)
    plot_func.plot_response(theta_full, tau_full_init, tau_full_desired, tau_full_desired, theta_range=[-90, 90], just_init=False)

## Measure Fy - double hinge

In [ ]:
if problem == 'Fy':
    L=1
    x = -0.28
    y = 1.8
    theta1, theta2 = theta_from_xy(x, y, -init_buckle[0])
    print(np.degrees(theta1), np.degrees(theta2))
    tau1 = tau_k(theta1, [1, 1], Variabs)[0]
    tau2 = tau_k(theta2, [1, 1], Variabs)[1]
    print(tau1, tau2)
    Fy1 = Fy(theta1, theta2, tau1, tau2)
    print(Fy1)

    # Reachability check
    reach_min, reach_max = 0, 2*L
    r = np.hypot(x, y)
    if r < reach_min - 1e-9 or r > reach_max + 1e-9:
        print("WARNING: Target point is outside reachable workspace. The diagram uses clipped IK (c2 clipped).")

    plot_arm(x, y, theta1, theta2)

In [ ]:
if problem == 'Fy':
    for i in range(iterations):

        ## Measure
        Supervisor.set_pos(i)
        State.calc_tau(Supervisor.theta, Variabs)

        ## Loss
        Supervisor.calc_loss(State.tau)

        ## Update
        Supervisor.calc_input_update(State, Supervisor, Variabs)
        State.calc_tau(Supervisor.input_update, Variabs)
        State.evolve_material(Supervisor, Variabs)

    Supervisor.loss_MSE_norm_in_t = Supervisor.loss_MSE_in_t / (Variabs.k_bar * Variabs.theta_bar)